# Getting started with prompt flow code-first experience

**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Write LLM application using notebook and visualize the trace of your application.
- Convert the application into a flow and batch run against multi lines of data.


## 0. Install dependent packages

In [ ]:
%%capture --no-stderr
%pip install -r ./requirements.txt

## 1. Trace your application with promptflow

Assume we already have a python function that calls OpenAI API. 

In [ ]:
with open("llm.py") as fin:
    print(fin.read())

Note: before running below cell, please configure required environment variable `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_API_BASE` by create an `.env` file. Please refer to [.env.example](.env.example) as an template.

In [ ]:
# control the AOAI deployment (model) used in this example
deployment_name = "text-davinci-003"

In [ ]:
from llm import my_llm_tool

# pls configure `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_API_BASE` environment variables first
result = my_llm_tool(
    prompt="Write a simple Hello, world! program that displays the greeting message when executed.",
    deployment_name=deployment_name,
)
result

### Visualize trace by using promptflow.start_trace

In [ ]:
# TODO: when public available, change to from promptflow import start_trace
from promptflow._trace._start_trace import start_trace

# start a trace session, and print a url for user to check trace
start_trace()

Note we add `@trace` in the `my_llm_tool` function, re-run below cell will collect a trace in trace UI.

In [ ]:
# rerun the function, which will
result = my_llm_tool(
    prompt="Write a simple Hello, world! program that displays the greeting message when executed.",
    deployment_name=deployment_name,
)
result

Now, let's add another layer of function call. In [flow.py](flow.py) there is a function called `flow_entry`, which calls a new function called `load_prompt` and previous `my_llm_tool` function.

In [ ]:
# show the flow.py content
with open("flow.py") as fin:
    print(fin.read())

In [ ]:
# call the flow entry function
from flow import flow_entry

result = flow_entry("Java Hello, world!")
result

### Eval the result 

In [ ]:
# eval function: TODO
# from eval_flow import eval
# eval(answer= result["anser"], ground_truth='abc')

## 2. Batch run the function as flow with multi-line data

Create a [flow.dag.yaml](flow.dag.yaml) file to define a flow which entry pointing to a python function.


In [ ]:
# show the flow.dag.yaml content
with open("flow.dag.yaml") as fin:
    print(fin.read())

### Batch run with a data file (with multiple lines of test data)


In [ ]:
from promptflow import PFClient

flow = "."  # path to the flow directory
data = "./data.jsonl"  # path to the data file

# create run with the flow and data
pf = PFClient()
base_run = pf.run(flow=flow, data=data, stream=True)

In [ ]:
details = pf.get_details(base_run)
details.head(10)

## 3. Evaluate your flow
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which usually using LLM assert the produced output matches certain expectation. 

### Run evaluation on the previous batch run
The **base_run** is the batch run we completed in step 2 above, for web-classification flow with "data.jsonl" as input.

In [ ]:
# TODO using asserts

# eval_flow = "./eval_flow.dag.yaml"

# eval_run = pf.run(
#     flow=eval_flow,
#     data="./data.jsonl",  # path to the data file
#     run=base_run,  # specify base_run as the run you want to evaluate
#     column_mapping={
#         "groundtruth": "${data.answer}",
#         "prediction": "${run.outputs.category}",
#     },  # map the url field from the data to the url input of the flow
#     stream=True,
# )

In [ ]:
# details = pf.get_details(eval_run)
# details.head(10)

In [ ]:
# import json
# metrics = pf.get_metrics(eval_run)
# print(json.dumps(metrics, indent=4))

In [ ]:
# pf.visualize([base_run, eval_run])

# Next Steps

By now you've successfully run your first prompt flow and even did evaluation on it. That's great!

You can check out more examples:
- [Basic Chat](../../chat/chat-basic-code-first/): demonstrates how to create a chatbot that can remember previous interactions and use the conversation history to generate next message.